In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from src.data.data_loader import load_all_data
#df_raw = load_all_data()

import json

file = "../.data/raw/embeddings_generated_comments_slack_256.jsonl"
with open(file) as f:
    data_raw = [json.loads(line) for line in f]

In [197]:
import pandas as pd

df_raw = pd.DataFrame(data_raw)
df_raw

,contract_name,file_path,contract_address,language,class_name,class_code,class_documentation,class_documentation_type,func_name,func_code,...,output,generated,target,truncated,duration,bleu_score,generated_pooled_avg,generated_pooled_max,func_code_pooled_avg,func_code_pooled_max
0,TheCoinBBToken,TheCoinBBToken.sol,0x9d365b5dad49b4277aa444e523c5cc789d3f6959,Solidity,TheCoinBBToken,contract TheCoinBBToken {\r\n //币名字\r\n ...,,,TheCoinBBToken,"function TheCoinBBToken(uint256 initialSupply,...",...,/* Constructor */\r\n function ERC20Token(s...,"\r\n function ERC20Token(string _name, stri...","function TheCoinBBToken(uint256 initialSupply,...",False,7.300967,1.397416e-231,"[0.18231993913650513, 0.22477486729621887, 0.6...","[2.4258086681365967, 2.93768048286438, 2.80093...","[0.7295137047767639, 0.38593432307243347, 0.01...","[5.402975082397461, 2.5263583660125732, 4.1556..."
1,Keep3rV1,Keep3rV1.sol,0x5bab9eec479b1991455a89562a2cc7c274c5f95e,Solidity,Keep3rV1,contract Keep3rV1 is ReentrancyGuard {\r\n ...,,,isBondedKeeper,"function isBondedKeeper(address keeper, addres...",...,/**\r\n * @notice confirms if the current keep...,"\r\n function isMinKeeper(address keeper, a...","function isBondedKeeper(address keeper, addres...",False,6.953809,8.447944e-01,"[0.28921324014663696, 0.5198975205421448, 0.89...","[4.014957427978516, 3.8795106410980225, 5.3767...","[0.09057445824146271, 0.20432017743587494, 0.7...","[4.374789714813232, 3.376408338546753, 5.30213..."
2,Cellpinda,Cellpinda.sol,0x5f4a339fd4c64f8bb34935efa7a638b92ae82e9a,Solidity,SafeMath,library SafeMath {\r\n\r\n /**\r\n * @dev Mu...,/**\r\n * @title SafeMath\r\n * @dev Math oper...,NatSpecMultiLine,div,"function div(uint256 a, uint256 b) internal pu...",...,/**\r\n * @dev Integer division of two numbers...,"\r\nfunction div(uint256 a, uint256 b) interna...","function div(uint256 a, uint256 b) internal pu...",False,3.792178,9.139438e-01,"[0.007728193886578083, 0.37208354473114014, -0...","[2.3015894889831543, 3.829735279083252, 3.3693...","[-0.08193076401948929, 0.2569873631000519, -0....","[2.4103784561157227, 2.8426284790039062, 3.375..."
3,Tccchain,Tccchain.sol,0x91507ea472707e6587f359e0f71846ec097aefc7,Solidity,Tccchain,contract Tccchain {\r\n // Public variables...,,,_transfer,"function _transfer(address _from, address _to,...",...,"/**\r\n * Internal transfer, only can be calle...","\r\n \r\nfunction _transfer(address _from, add...","function _transfer(address _from, address _to,...",False,14.390161,9.474258e-01,"[0.450160950422287, 0.2462613582611084, 0.2296...","[3.338660955429077, 3.0911128520965576, 4.4330...","[0.5586769580841064, -0.056083500385284424, 0....","[3.369008779525757, 3.1738884449005127, 4.3181..."
4,ELB1,ELB1.sol,0x109e726df6f2fe1aaf0ca306e1582d305614075b,Solidity,IERC20,interface IERC20 {\r\n //function transfer(...,,,totalSupply,function totalSupply() external view returns (...,...,"//function transfer(address to, uint256 value)...","\n//event Transfer(address indexed from, addre...",function totalSupply() external view returns (...,False,6.811126,0.000000e+00,"[0.4172349274158478, 0.8019665479660034, -0.07...","[4.429458141326904, 3.211848735809326, 2.63131...","[1.1170274019241333, 0.35891929268836975, 0.02...","[2.195232391357422, 1.2374753952026367, 1.2371..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,DerpDAO,DerpDAO.sol,0x861c840d1cc29d90d84f6c18d1ee6ec372461055,Solidity,SafeCast,library SafeCast {\r\n /**\r\n *...,/**\r\n * @dev Wrappers over Solidity's uintXX...,NatSpecMultiLine,toUint128,function toUint128(uint256 value) internal pur...,...,/**\r\n * @dev Returns the downcasted uint128 ...,\r\nfunction toUint128(uint256 value) internal...,function toUint128(uint256 value) internal pur...,False,2.388531,5.915464e-01,"[-0.04910701513290405, -0.34202733635902405, 0...","[2.2722420692443848, 2.0409727096557617, 4.711...","[-0.26368406414985657, -0.1918938010931015, 0....","[2.272425651550293, 2.

In [198]:
df_raw = df_raw.drop_duplicates(subset=['func_documentation'])

In [212]:
import numpy as np
X = np.stack(df_raw["generated_pooled_avg"])

In [213]:
from sklearn.cluster import DBSCAN

# cluster each parent cluster

parent = DBSCAN(eps=0.5, min_samples=3)

parent_labels = parent.fit_predict(X)

In [214]:
import pandas as pd
df = pd.DataFrame(X)
df['label'] = parent_labels
#df = df.reset_index()
clusters = df.groupby('label', as_index=False, group_keys=True)
#groups = {k: v.values for k, v in df.groupby('label')}
#print(groups)

In [215]:
def cluster_sub_clusters(cluster):
    if cluster.name == -1:
        print("Skipping noise cluster")
        cluster["sub_label"] = -1
        return cluster

    #if cluster.name == -1:
    #    print("Skipping noise cluster")
    #    return
    
    algorithm = DBSCAN(eps=0.5, min_samples=3)
    sub_clusters = algorithm.fit(cluster.drop('label', axis=1).to_numpy())
    cluster["sub_label"] = sub_clusters.labels_.tolist()
    return cluster

clusters = clusters.apply(cluster_sub_clusters)


Skipping noise cluster


In [216]:
clusters = clusters.droplevel(0).sort_index()

In [217]:
clusters.label.value_counts().shape

(38,)

In [218]:
clusters.sub_label.value_counts().shape

(2,)

In [219]:
clusters

,0,1,2,3,4,5,6,7,8,9,...,4088,4089,4090,4091,4092,4093,4094,4095,label,sub_label
0,0.182320,0.224775,0.657518,-0.115372,-0.205114,0.301772,-0.943246,0.100650,0.415164,0.343421,...,-0.102552,-0.007069,-0.164095,0.434412,-0.470327,0.327597,0.165317,-0.161877,-1,-1
1,0.289213,0.519898,0.898107,-0.485910,-0.363001,0.093175,0.073298,1.265028,0.304592,0.152461,...,-0.171547,0.484903,-0.088196,-1.347180,0.885693,-0.244743,0.228073,0.932032,-1,-1
2,0.007728,0.372084,-0.149015,0.188889,-0.498045,-0.242688,-0.249023,-0.237050,0.652002,0.499453,...,-0.317190,0.385600,-0.738925,0.292483,0.356881,0.110507,0.283217,0.361806,-1,-1
3,0.450161,0.246261,0.229694,-0.039942,-0.612668,-0.239986,-0.711562,0.820301,0.160995,0.829683,...,-0.126881,0.177843,-0.403387,-0.154788,0.626566,0.342299,0.072371,0.358573,-1,-1
4,0.417235,0.801967,-0.077508,-0.373618,0.031005,0.057182,-0.305771,0.569606,0.145428,0.093630,...,0.056662,0.211487,-0.425525,0.229723,0.208002,0.461347,0.317514,-0.154114,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4690,-0.172539,0.377824,0.753085,0.174732,0.491238,-0.027144,0.134557,-0.047586,1.222642,-0.180025,...,0.161317,0.221778,0.475998,-0.122619,-0.756117,0.080759,0.683771,-0.223163,-1,-1
4691,0.735223,-0.076391,-0.020506,-0.402833,-1.217645,-0.181421,0.346839,0.291618,-0.080242,-0.068818,...,0.026136,0.012785,0.636524,-0.354960,0.202229,1.162865,0.322126,0.114911,-1,-1
4692,-0.224187,0.441177,-0.104753,-0.272141,0.432681,-0.117448,-0.717770,1.090934,0.191735,0.431134,...,0.339840,-0.291522,-0.754401,0.057724,0.856309,0.575702,0.287441,0.058210,-1,-1
4693,0.140080,0.888245,0.990420,0.115861,-0.353565,0.174048,-0.571221,-0.229588,0.035642,-0.147859,...,-0.274296,0.748596,0.182218,0.099332,0.064013,0.720326,-0.096590,-0.238026,-1,-1


In [220]:
clusters["sub_label"].value_counts()

sub_label
-1    4533
 0     162
Name: count, dtype: int64

In [221]:
import numpy as np
import random

vectors = clusters.drop(['label', 'sub_label'], axis=1).to_numpy()

metadata = np.stack(
    [
        df_raw["func_documentation"].apply(lambda x: x.encode("unicode_escape").decode("utf-8")).values,
        df_raw["func_code"].apply(lambda x: x.encode("unicode_escape").decode("utf-8")).values,
        df_raw['bleu_score'].values.astype(str),
        clusters['label'].values.astype(str),
        clusters['sub_label'].values.astype(str),
    ],
    axis=1
).tolist()


In [193]:
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%b-%d_%H-%M-%Ss")

writer = SummaryWriter(log_dir='../.data/runs/' + dt_string + '-max')
writer.add_embedding(mat=vectors, metadata=metadata, metadata_header=['func_documentation', 'func_code', 'bleu_score', 'cluster', 'sub_cluster'])
writer.close()

In [222]:
df_raw["cluster"] = clusters["label"].values
df_raw["sub_cluster"] = clusters["sub_label"].values

/var/folders/bg/v12j1gk96gz44lcngd9z6ddh0000gn/T/ipykernel_87070/3338481944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["cluster"] = clusters["label"].values
/var/folders/bg/v12j1gk96gz44lcngd9z6ddh0000gn/T/ipykernel_87070/3338481944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["sub_cluster"] = clusters["sub_label"].values


In [223]:
df_raw

,contract_name,file_path,contract_address,language,class_name,class_code,class_documentation,class_documentation_type,func_name,func_code,...,target,truncated,duration,bleu_score,generated_pooled_avg,generated_pooled_max,func_code_pooled_avg,func_code_pooled_max,cluster,sub_cluster
0,TheCoinBBToken,TheCoinBBToken.sol,0x9d365b5dad49b4277aa444e523c5cc789d3f6959,Solidity,TheCoinBBToken,contract TheCoinBBToken {\r\n //币名字\r\n ...,,,TheCoinBBToken,"function TheCoinBBToken(uint256 initialSupply,...",...,"function TheCoinBBToken(uint256 initialSupply,...",False,7.300967,1.397416e-231,"[0.18231993913650513, 0.22477486729621887, 0.6...","[2.4258086681365967, 2.93768048286438, 2.80093...","[0.7295137047767639, 0.38593432307243347, 0.01...","[5.402975082397461, 2.5263583660125732, 4.1556...",-1,-1
1,Keep3rV1,Keep3rV1.sol,0x5bab9eec479b1991455a89562a2cc7c274c5f95e,Solidity,Keep3rV1,contract Keep3rV1 is ReentrancyGuard {\r\n ...,,,isBondedKeeper,"function isBondedKeeper(address keeper, addres...",...,"function isBondedKeeper(address keeper, addres...",False,6.953809,8.447944e-01,"[0.28921324014663696, 0.5198975205421448, 0.89...","[4.014957427978516, 3.8795106410980225, 5.3767...","[0.09057445824146271, 0.20432017743587494, 0.7...","[4.374789714813232, 3.376408338546753, 5.30213...",-1,-1
2,Cellpinda,Cellpinda.sol,0x5f4a339fd4c64f8bb34935efa7a638b92ae82e9a,Solidity,SafeMath,library SafeMath {\r\n\r\n /**\r\n * @dev Mu...,/**\r\n * @title SafeMath\r\n * @dev Math oper...,NatSpecMultiLine,div,"function div(uint256 a, uint256 b) internal pu...",...,"function div(uint256 a, uint256 b) internal pu...",False,3.792178,9.139438e-01,"[0.007728193886578083, 0.37208354473114014, -0...","[2.3015894889831543, 3.829735279083252, 3.3693...","[-0.08193076401948929, 0.2569873631000519, -0....","[2.4103784561157227, 2.8426284790039062, 3.375...",-1,-1
3,Tccchain,Tccchain.sol,0x91507ea472707e6587f359e0f71846ec097aefc7,Solidity,Tccchain,contract Tccchain {\r\n // Public variables...,,,_transfer,"function _transfer(address _from, address _to,...",...,"function _transfer(address _from, address _to,...",False,14.390161,9.474258e-01,"[0.450160950422287, 0.2462613582611084, 0.2296...","[3.338660955429077, 3.0911128520965576, 4.4330...","[0.5586769580841064, -0.056083500385284424, 0....","[3.369008779525757, 3.1738884449005127, 4.3181...",-1,-1
4,ELB1,ELB1.sol,0x109e726df6f2fe1aaf0ca306e1582d305614075b,Solidity,IERC20,interface IERC20 {\r\n //function transfer(...,,,totalSupply,function totalSupply() external view returns (...,...,function totalSupply() external view returns (...,False,6.811126,0.000000e+00,"[0.4172349274158478, 0.8019665479660034, -0.07...","[4.429458141326904, 3.211848735809326, 2.63131...","[1.1170274019241333, 0.35891929268836975, 0.02...","[2.195232391357422, 1.2374753952026367, 1.2371...",-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9982,FeeDistributor,FeeDistributorHelpers.sol,0xc2451b67a3ffd623c7607943d57b68213111ec0c,Solidity,FeeDistributorHelpers,contract FeeDistributorHelpers {\n using Sa...,// Helpers to do some conversions,LineComment,_fromATokenV1,function _fromATokenV1(address _atoken) intern...,...,function _fromATokenV1(address _atoken) intern...,False,15.551116,0.000000e+00,"[-0.17253926396369934, 0.37782374024391174, 0....","[5.148400783538818, 2.8747057914733887, 4.2052...","[0.06665542721748352, 0.1127094179391861, 0.64...","[1.634053111076355, 2.3592753410339355, 2.9337...",-1,-1
9989,InvestmentPool,InvestmentPool.sol,0x7d70447241be7442ae08e96d7ef1d0c13d727f29,Solidity,CancellableInvestmentPool,contract CancellableInvestmentPool is BaseInve...,/**\r\n * @title CancellableInvestmentPool\r\n...,NatSpecMultiLine,_preValidateFinalization,function _preValidateFinalization() internal {...,...,function _preValidateFinalization() internal {...,False,4.537421,7.919884e-232,"[0.7352228164672852, -0.07639122009277344, -0....","[3.4502975940704346, 1.968425989151001, 2.4874...","[0.46821817

In [224]:
df_raw.to_pickle("dbscan_unique_average.pkl")